<a href="https://colab.research.google.com/github/ComputerWizard2/whisper_base/blob/main/whisper_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 环境配置

In [1]:
pip install kagglehub==0.3.13 datasets==3.2.0 librosa==0.10.2.post1 peft==0.14.0 torchaudio==2.6.0 ffmpeg-python==0.2.0 transformers==4.51.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 下载模型

In [2]:
import kagglehub
path = kagglehub.dataset_download("tenffe/common-voice-zh-cn")
print("Path to dataset files:", path)

100%|██████████| 1.84G/1.84G [00:29<00:00, 67.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tenffe/common-voice-zh-cn/versions/1


## 下载测试数据

In [3]:
import kagglehub
path = kagglehub.dataset_download("shawnchile/voice-female-yuri")
print("Path to dataset files:", path)

100%|██████████| 21.3M/21.3M [00:00<00:00, 60.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shawnchile/voice-female-yuri/versions/1


## 下载预训练模型


```
cd pretrained_model
wget https://dtse-mirrors.obs.cn-north-4.myhuaweicloud.com/case/0002/whisper_base.tar.gz
tar -zxvf whisper_base.tar.gz


```



 ## 处理数据

In [1]:
import os
from datasets import load_from_disk, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
# 配置参数
model_name_or_path = "/content/pretrained_model/whisper_base"
language = "chinese"
language_abbr = "zh"
task = "transcribe"
dataset_name = "/content/data/1/common_voice_zh_CN"
prepared_dataset_path = "/content/pretrained_model/prepared_common_voice"
# 加载数据集
common_voice = load_from_disk(dataset_name)
# 选择前1000个样本
common_voice['train'] = common_voice['train'].select(range(100))
if 'test' in common_voice:
    common_voice['test'] = common_voice['test'].select(range(100))
if 'validation' in common_voice:
    common_voice['validation'] = common_voice['validation'].select(range(100))
# 移除不需要的列
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)
# 将采样率改为16kHz
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
# 预处理音频数据和对应的文本
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch
# 加载特征提取器和分词器
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)
# 预处理数据集
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)
# 保存预处理后的数据集
common_voice.save_to_disk(prepared_dataset_path)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

### 更新torch包的版本

In [5]:
!pip uninstall -y torchvision torch torchaudio
!pip install kagglehub==0.3.13 datasets==3.2.0 librosa==0.10.2.post1 peft==0.14.0 torch==2.6.0 torchaudio==2.6.0 torchvision==0.21.0 ffmpeg-python==0.2.0 transformers==4.51.3

Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchaudio 2.6.0
Uninstalling torchaudio-2.6.0:
  Successfully uninstalled torchaudio-2.6.0
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchaudio-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (6.6 kB)
Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl (766.6 MB)
Using cached torchaudio-2.6.0-cp312-cp312-manylinux1_x86_64.whl (3.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.2 MB/s eta 0:00:00


## 设置LoRA参数

In [2]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, AutoConfig
from peft import LoraConfig, get_peft_model
import os
import torch
# 禁用 safetensors
os.environ["TRANSFORMERS_SAFE_TENSORS"] = "false"
model_name_or_path = "/content/pretrained_model/whisper_base" # 加载模型配置
config = AutoConfig.from_pretrained(model_name_or_path)
# 手动加载模型文件
state_dict = torch.load(f"{model_name_or_path}/pytorch_model.bin", weights_only=True)
# 初始化模型
model = WhisperForConditionalGeneration(config)
# 尝试加载 state_dict，允许忽略缺失的键
model.load_state_dict(state_dict, strict=False)
# 打印模型信息
print(model)
# 定义 LoRA 配置
config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05)
# 获取 LoRA 模型
model = get_peft_model(model, config)
# 打印可训练参数
model.print_trainable_parameters()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=False)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          

# 基于 LoRA 微调 Whisper 模型实现中文语音识别

## 项目简介

本项目旨在利用 LoRA (Low-Rank Adaptation) 技术对 OpenAI 的 Whisper `base` 模型进行微调，以实现高效的中文语音识别 (ASR)。通过在 Common Voice 中文数据集上进行训练，并使用 LoRA 显著减少了可训练参数数量，从而加速了训练过程并降低了计算资源消耗。最终模型将用于对测试音频文件进行转录。

## 环境配置

项目依赖以下库：

-   `kagglehub==0.3.13`
-   `datasets==3.2.0`
-   `librosa==0.10.2.post1`
-   `peft==0.14.0`
-   `torch==2.6.0`
-   `torchaudio==2.6.0`
-   `torchvision==0.21.0`
-   `ffmpeg-python==0.2.0`
-   `transformers==4.51.3`

可以使用以下命令进行安装：

```bash
!pip install kagglehub==0.3.13 datasets==3.2.0 librosa==0.10.2.post1 peft==0.14.0 torch==2.6.0 torchaudio==2.6.0 torchvision==0.21.0 ffmpeg-python==0.2.0 transformers==4.51.3
```

**注意**: 如果遇到 `torch_npu` 相关错误，请确保使用 `torch` 和 `torchvision` 的兼容版本，并优先使用 CUDA (GPU) 或 CPU 设备。

## 数据下载

### 训练数据

本项目使用 Kaggle 上的 [Common Voice 中文数据集](https://www.kaggle.com/datasets/tenffe/common-voice-zh-cn) 进行模型微调。

```python
import kagglehub
path = kagglehub.dataset_download("tenffe/common-voice-zh-cn")
print("Path to dataset files:", path)
```

### 测试数据

使用 Kaggle 上的 [voice-female-yuri 数据集](https://www.kaggle.com/datasets/shawnchile/voice-female-yuri) 进行推理测试。

```python
import kagglehub
path = kagglehub.dataset_download("shawnchile/voice-female-yuri")
print("Path to dataset files:", path)
```

## 预训练模型下载

下载 Whisper `base` 模型的预训练权重。

```bash
!mkdir -p pretrained_model
!cd pretrained_model && wget https://dtse-mirrors.obs.cn-north-4.myhuaweicloud.com/case/0002/whisper_base.tar.gz
!cd pretrained_model && tar -zxvf whisper_base.tar.gz
```

## 数据预处理

加载 Common Voice 数据集，并进行音频采样率调整、特征提取和文本分词。

```python
import os
from datasets import load_from_disk, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# 配置参数
model_name_or_path = "/content/pretrained_model/whisper_base"
language = "chinese"
task = "transcribe"
dataset_name = "/content/data/1/common_voice_zh_CN"
prepared_dataset_path = "/content/pretrained_model/prepared_common_voice"

# ... (省略具体实现代码，详见 notebook)
```

## 设置 LoRA 参数

配置 LoRA 适配器，将其应用于 Whisper 模型，以实现参数高效微调。

```python
from transformers import WhisperForConditionalGeneration, AutoConfig
from peft import LoraConfig, get_peft_model
import os
import torch

# 禁用 safetensors
os.environ["TRANSFORMERS_SAFE_TENSORS"] = "false"
model_name_or_path = "/content/pretrained_model/whisper_base"

# ... (省略具体实现代码，详见 notebook)

# 定义 LoRA 配置
config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05)
model = get_peft_model(model, config)
print(model.print_trainable_parameters())
```

## 模型训练

使用 `Seq2SeqTrainer` 在预处理后的数据集上训练带有 LoRA 适配器的 Whisper 模型。

```python
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model
import torch

# ... (省略 DataCollatorSpeechSeq2SeqWithPadding 定义及其他参数配置，详见 notebook)

# 设置 Trainer 参数 (已禁用 wandb 报告)
training_args = Seq2SeqTrainingArguments(
    # ...
    report_to=["none"], # Disable wandb reporting
)

# 创建 Trainer 并开始训练
# ... (省略具体实现代码，详见 notebook)

# 保存 LoRA 模型
model.save_pretrained("/content/pretrained_model/checkpoint_1000_samples")
```

## 模型推理与语音识别

加载微调后的 LoRA 模型，并使用 `AutomaticSpeechRecognitionPipeline` 对测试音频文件进行语音转录。

```python
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor
from peft import PeftModel, PeftConfig
from transformers import AutomaticSpeechRecognitionPipeline
import torch
import os
import json

# 配置参数
device = "cuda" if torch.cuda.is_available() else "cpu"
peft_model_id = "/content/pretrained_model/checkpoint_1000_samples"
audio_directory = "/content/data/wavs/1/wavs" # 测试音频路径
output_directory = "/content/output" # 输出结果路径

# ... (省略具体实现代码，详见 notebook)

# 将结果保存到 JSON 文件中
output_file = os.path.join(output_directory, "transcription_results.json")
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
print(f"Results saved to {output_file}")
```

## 结果

转录结果将保存到 `/content/output/transcription_results.json` 文件中。


## 训练模型

In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
from datasets import load_from_disk  # 导入 load_from_disk 函数
import torch
from peft import LoraConfig, get_peft_model, PeftModel
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    def __init__(self, processor: Any):
        self.processor = processor
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch
# 配置参数
model_name_or_path = "/content/pretrained_model/whisper_base"
language = "chinese"
task = "transcribe"
prepared_dataset_path = "/content/pretrained_model/prepared_common_voice"   # 预处理后的数据集路径
# 设置设备为NPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 加载特征提取器和分词器
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)
# 加载预处理后的数据集
common_voice = load_from_disk(prepared_dataset_path)
# 定义 LoRA 配置
lora_config = LoraConfig(
    r=8,  # LoRA attention dimension
    lora_alpha=32,  # Alpha scaling
    target_modules=["q_proj", "v_proj"],  # Target modules to apply LoRA
    lora_dropout=0.1,  # Dropout probability for LoRA layers
)
# 加载模型
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)
# 将模型移动到NPU
model = model.to(device)
# 将 LoRA 配置应用到模型
model = get_peft_model(model, lora_config)
# 设置 Trainer 参数
training_args = Seq2SeqTrainingArguments(
    output_dir="your-name/int8-whisper-base-v2-asr",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=2e-3,
    warmup_steps=50,
    num_train_epochs=3,
    eval_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=32,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,
    label_names=["labels"],
    no_cuda=False,  # 允许使用 NPU/GPU
    use_cpu=False,
    report_to=["none"], # Disable wandb reporting
)
# 创建数据收集器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
# 创建 Trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"].select(range(100)),  # 仅使用前100个样本进行训练
    eval_dataset=common_voice["test"].select(range(100)),  # 仅使用前100个样本进行验证
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
# 开始训练
model.config.use_cache = False
trainer.train()
# 保存 LoRA 模型
model.save_pretrained("/content/pretrained_model/checkpoint_1000_samples")

Using device: cuda


/tmp/ipython-input-1498734386.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,No log,1.967222
2,No log,1.419844
3,No log,1.263087


## 推理模型实现语音识别

In [13]:
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor
from peft import PeftModel, PeftConfig
from transformers import AutomaticSpeechRecognitionPipeline
import torch
import os
import json
# 配置参数
device = "cuda" if torch.cuda.is_available() else "cpu"
peft_model_id = "/content/pretrained_model/checkpoint_1000_samples"
language = "chinese"
task = "transcribe"
output_directory = "/content/output"  # 输出文件夹路径
# 确保输出目录存在
os.makedirs(output_directory, exist_ok=True)
# 加载 LoRA 模型配置
peft_config = PeftConfig.from_pretrained(peft_model_id)
# 加载基础模型
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path
)
# 基于基础模型和 LoRA 模型路径构建 LoRA 模型
model = PeftModel.from_pretrained(model, peft_model_id)
# 将 PeftModel 转换为普通的 WhisperForConditionalGeneration 模型
model = model.merge_and_unload()
# 加载分词器（处理文本）和处理器（封装特征抽取器和分词器）、以及特征抽取器（处理音频）
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
# 构建一个 pipeline 进行测试
pipeline = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor, device=device)
# 测试音频文件路径
audio_directory = "/content/data/wavs/1/wavs"
# 获取目录下的所有 .wav 文件
audio_files = [os.path.join(audio_directory, f) for f in os.listdir(audio_directory) if f.endswith('.wav')]
# 只处理前100个文件
audio_files = audio_files[:100]
# 存储结果的列表
results = []
# 遍历所有音频文件并进行语音识别
for audio_file in audio_files:
    with torch.no_grad():
        text = pipeline(audio_file, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
        results.append({"file": audio_file, "text": text})
        print(f"Transcribed text for {audio_file}: {text}")
# 将结果保存到 JSON 文件中
output_file = os.path.join(output_directory, "transcription_results.json")
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
print(f"Results saved to {output_file}")

Device set to use cuda
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcribed text for /content/data/wavs/1/wavs/0013095.wav: 完 算完好了吗


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcribed text for /content/data/wavs/1/wavs/0013090.wav: 一号为死亥逝严逝这个那就这么一块儿有在定了


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


...

Transcribed text for /content/data/wavs/1/wavs/0013106.wav: 可这并不是因为他赶猫了。


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcribed text for /content/data/wavs/1/wavs/0013086.wav: 我可是从长单树上的时候就开始做菜了


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcribed text for /content/data/wavs/1/wavs/0013073.wav: 所以今天的视频是猫血猫和水主用片
Results saved to /content/output/transcription_results.json
